In [1]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

### **런타임 Config 변경**
런타임에서도 retriever 의 속성을 변경할 수 있습니다. ConfigurableField 클래스를 사용하여 가능합니다. 
<br>

weights 매개변수를 ConfigurableField 객체로 정의합니다.
- 필드의 ID는 "ensemble_weights"로 설정합니다.

In [2]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

In [3]:

from langchain_core.runnables import ConfigurableField

In [4]:
doc_list = [
    "I like apples",
    "I like apple company",
    "I like apple's iphone",
    "Apple is my favorite company",
    "I like apple's ipad",
    "I like apple's macbook",
]

In [9]:
# bm25 retriever
bm25_retriever = BM25Retriever.from_texts(doc_list)
bm25_retriever.k = 1

In [ ]:
# aiss retriever
embedding = OpenAIEmbeddings(api_key=key)
faiss_vectorstore = FAISS.from_texts(
    doc_list,
    embedding,
)

faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 1})

In [13]:
ensemble_retriever = EnsembleRetriever(

    retrievers=[                        # 리트리버 목록을 설정
        bm25_retriever,                 # bm25_retriever
        faiss_retriever                 # faiss_retriever
    ]
).configurable_fields(
    weights=ConfigurableField(
        id="ensemble_weights",          # 검색 매개변수의 id를 설정
        name="Ensemble Weights",        # 검색 매개변수의 이름을 설정
        description="Ensemble Weights"  # 검색 매개변수에 대한 설명을 작성
    )
)

검색 시 `config` 매개변수를 통해 검색 설정을 지정합니다. <br>
`ensemble_weights` 옵션의 가중치를 [1, 0]으로 설정해서 모든 검색 결과의 가중치가 BM25 retriever 에 더 많이 부여 되도록 합니다. <br>

In [14]:
config = {"configurable": {"ensemble_weights": [1, 0]}}

In [15]:
# config 매개변수를 사용하여 검색 설정을 지정합니다.
docs = ensemble_retriever.invoke("my favorite fruit is apple", config=config)

In [16]:
print(docs)

[Document(metadata={}, page_content='Apple is my favorite company'), Document(metadata={}, page_content='I like apples')]


검색시 모든 검색 결과의 가중치가 FAISS retriever 에 더 많이 부여 되도록 합니다.

In [17]:
# config 매개변수를 사용하여 검색 설정을 지정합니다.
config = {"configurable": {"ensemble_weights": [0, 1]}}

In [18]:
# config 매개변수를 사용하여 검색 설정을 지정합니다.
docs = ensemble_retriever.invoke("my favorite fruit is apple", config=config)

In [19]:
print(docs)

[Document(metadata={}, page_content='I like apples'), Document(metadata={}, page_content='Apple is my favorite company')]
